In [1]:
import argparse
import sys
import os
import time
import copy

import tensorflow as tf
import pandas as pd

import dataset
# from models.conv2_dense2_dropout import Model
from models.dense3 import Model

from helpers.history import history
#from helpers.gpu_utils import validate_batch_size_for_multi_gpu
from helpers.softmax_cross_entropy_trainer import create_model_fn

### Get the history and the runtime context 

In [2]:
localtime = time.asctime(time.localtime(time.time()))
user = os.environ.get('USER', os.environ.get('USERNAME', 'anonymous'))
print("\n\n")
print("Welcome, %s, it's %s, and you'll be working with Tensorflow version %s" % (user, localtime, tf.__version__))
print("\n")
history.experiments.tail(10)




Welcome, wgiersche, it's Tue May  1 09:11:15 2018, and you'll be working with Tensorflow version 1.8.0




,batch_size,data_dir,model_dir,multi_gpu,train_epochs,user,timestamp,localtime,accuracy,steps
5,64,/var/ellie/data/mnist_fashion,/tmp/mnist_model,0,10,wgiersche,1.525017e+09,Sun Apr 29 17:50:42 2018,NaN,NaN
6,64,/var/ellie/data/mnist_fashion,/tmp/mnist_model,0,10,wgiersche,1.525018e+09,Sun Apr 29 18:05:46 2018,NaN,NaN
7,256,/var/ellie/data/mnist_fashion,C:\tmp\mnist_model,0,6,wgi,1.525089e+09,Mon Apr 30 13:48:20 2018,0.9801,1410.0
8,64,/var/ellie/data/mnist_fashion,/tmp/mnist_model,0,10,wgiersche,1.525102e+09,Mon Apr 30 17:28:43 2018,NaN,NaN
9,64,/var/ellie/data/mnist_fashion,/tmp/mnist_model,0,10,wgiersche,1.525102e+09,Mon Apr 30 17:28:43 2018,NaN,NaN
10,64,/var/ellie/data/mnist_fashion,/tmp/mnist_model,0,10,wgiersche,1.525102e+09,Mon Apr 30 17:30:15 2018,NaN,NaN
11,64,/var/ellie/data/mnist_fashion,/tmp/mnist_model,0,10,wgiersche,1.525102e+09,Mon Apr 30 17:32:06 2018,NaN,NaN
12,256,/var/ellie/data/mnist_fashion,/tmp/mnist_model,0,4,wgiersche,1.525103e+09,Mon Apr 30 17:48:42 2018,0.8754,1880.0
13,256,/var/ellie/data/mnist_fashion,/tmp/mnist_model,0,4,wgiersche,1.525148e+09,Tue May 1 06:17:09 2018,0.8765,940.0
14,256,/var/ellie/data/mnist_fashion,/tmp/mnist_model,0,4,wgiersche,1.525149e+09,Tue May 1 06:36:35 2018,0.8801,940.0


### Create a new hyper-parameter record 

In [6]:
# Creating a new hyper-parameter record from a history entry
FLAGS = history.copy_from_record(2)

# Choose new parameters
FLAGS.train_epochs = 40
FLAGS.batch_size = 256
FLAGS.multi_gpu = True
FLAGS

batch_size                                256
data_dir        /var/ellie/data/mnist_fashion
model_dir                    /tmp/mnist_model
multi_gpu                                True
train_epochs                               40
user                                wgiersche
timestamp                         1.52516e+09
localtime            Tue May  1 09:11:54 2018
accuracy                                  NaN
steps                                     NaN
Name: 2, dtype: object

### Get to work!

In [4]:
# For the sake of this tutorial, we always start from scratch
!rm -rf /tmp/mnist_model

In [5]:
model_function = create_model_fn(
    lambda params: Model(params),
    tf.train.AdamOptimizer(),
    FLAGS)

INFO:tensorflow:Replicating the `model_fn` across [u'/device:GPU:0', u'/device:GPU:1'].  Variables are going to be placed on ['/CPU:0'].  Consolidation device is going to be /CPU:0.


In [8]:
#if FLAGS.multi_gpu:
#    validate_batch_size_for_multi_gpu(FLAGS.batch_size)

    # There are two steps required if using multi-GPU: (1) wrap the model_fn,
    # and (2) wrap the optimizer. The first happens here, and (2) happens
    # in the model_fn itself when the optimizer is defined.
#    model_function = tf.contrib.estimator.replicate_model_fn(
#        model_function, loss_reduction=tf.losses.Reduction.MEAN)

In [7]:
data_format = ('channels_first' if tf.test.is_built_with_cuda() else 'channels_last')

In [8]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=model_function,
    model_dir=FLAGS.model_dir,
    params={
        'data_format': data_format,
        'multi_gpu': FLAGS.multi_gpu
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4c4f842a90>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/mnist_model', '_global_id_in_cluster': 0, '_save_summary_steps': 100}


##### ```input_fn``` functions are a factories for ```DataSet```s

In [9]:
def train_input_fn():
    ds = dataset.training_dataset(FLAGS.data_dir)
    ds = ds.cache().shuffle(buffer_size=50000).\
        batch(FLAGS.batch_size).\
        repeat(FLAGS.train_epochs)
    return ds

In [10]:
def eval_input_fn():
    return dataset.test_dataset(FLAGS.data_dir).\
        batch(FLAGS.batch_size).\
        make_one_shot_iterator().get_next()

In [11]:
tensors_to_log = {'train_accuracy': 'train_accuracy'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=1000)

### Run the training and report the new hyper-parameters 

In [12]:
mnist_classifier.train(input_fn=train_input_fn, hooks=[logging_hook])
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
FLAGS.accuracy = eval_results['accuracy']
FLAGS.steps = eval_results['global_step']
history.report_experiment(FLAGS)

print('Evaluation results:\n\t%s' % eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_model/model.ckpt.
INFO:tensorflow:train_accuracy = 0.109375
INFO:tensorflow:loss = 2.4076366, step = 0
INFO:tensorflow:global_step/sec: 104.964
INFO:tensorflow:loss = 0.43944514, step = 100 (0.953 sec)
INFO:tensorflow:global_step/sec: 233.899
INFO:tensorflow:loss = 0.3955303, step = 200 (0.427 sec)
INFO:tensorflow:global_step/sec: 227.751
INFO:tensorflow:loss = 0.3076253, step = 300 (0.439 sec)
INFO:tensorflow:global_step/sec: 231.714
INFO:tensorflow:loss = 0.36498427, step = 400 (0.432 sec)
INFO:tensorflow:global_step/sec: 228.098
INFO:tensorflow:loss = 0.3281408, step = 500 (0.438 sec)
INFO:tensorflow:global_step/sec: 232.723
INFO:tensorflow:loss = 0.32624006, step = 600 (0.430 sec)
INFO:tensorf

In [13]:
FLAGS

batch_size                                256
data_dir        /var/ellie/data/mnist_fashion
model_dir                    /tmp/mnist_model
multi_gpu                                True
train_epochs                               40
user                                wgiersche
timestamp                         1.52516e+09
localtime            Tue May  1 09:11:54 2018
accuracy                               0.8918
steps                                    9400
Name: 2, dtype: object